# Inference notebook

In this notebook, we make predictions in segmenting defects in hazelnut images from the MVTec Anomaly Detection [dataset](https://www.mvtec.com/company/research/datasets/mvtec-ad).


First, import all the necessary libraries:

In [ ]:
import os
import tensorflow as tf

from omegaconf import OmegaConf

from processing.preprocessing import preprocess_data_from_images
from processing.postprocessing import encode_masks_to_rgb
from utils.dir_processing import clean_folder
from utils.utils import create_color_map, create_category_dict

from models import evaluate
from models.model_functions import load_model
from models.saving import load_params
from models.train_test.test import test_model
from models.saving import save_predictions

from sklearn.model_selection import train_test_split

All configuration parameters are defined in `env.yaml` file. These parameters include information about the location of the dataset and results folders, the architecture of the network, and its training parameters.

Here we will use a pretrained model called `fuzzy_ivory_macaque`.

In [ ]:
# load environment variables
cfg = OmegaConf.load('configs/env.yaml')

cfg.MODEL.model_name = "whiny_red_mastiff"

# used for inference only
categ_dict = create_category_dict(cfg.MODEL.categories)
color_map = create_color_map(cfg.MODEL.categories)

print('Class categories', categ_dict)

num_classes = len(cfg.MODEL.categories) + 1

Load model and weights:

In [ ]:
# use existing model with its proper name
model_name = cfg.MODEL.model_name

model = load_model(model_name)
load_params(model_name, cfg)

print(f'Model name is {model_name}')    
    
model.summary()

Here we load and preprocess images and masks from the dataset. We split the initial dataset into training and testing subsets. 

We will use only the test subset for inference.

In [ ]:
# === DATASET LOADING AND PREPROCESSING === #
X, y = preprocess_data_from_images(data_path = cfg.DIRS.data, 
                                   shape = eval(cfg.DATA.img_dims),
                                   categories = cfg.MODEL.categories)

#=== TRAIN/TEST SPLIT === #
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=cfg.DATA.test_split,
                                                    shuffle=cfg.TRAINING.shuffle,
                                                    random_state=cfg.DATA.seed)

print(f'Number of TRAIN images: {len(X_train)}')
print(f'Number of TEST images: {len(X_test)}')

Get predictions

In [ ]:
# == INFERENCE == #
y_pred = test_model(model, X_test, prediction_threshold=cfg.TRAINING.prediction_threshold)

Get confusion matrix

In [ ]:
# == Confusion matrices == #
confusion_classes, imgs_labels = evaluate.get_confusion_indices(y_test,
                                                                y_pred,
                                                                categories_dict = categ_dict,
                                                                pixel_thres = cfg.TRAINING.pixel_threshold,
                                                                meanIoU_threshold = cfg.TRAINING.iou_threshold)            
            
for class_name,confusion_matrix in confusion_classes.items():        
    evaluate.save_confusion_matrix(confusion_matrix, model_name, class_name, class_counter=None)

Save predictions sorting them into true positive (tp), true negative (tn), false positive (fp) and false negatives (fn) folders.

In [ ]:
# encode ground truth and prediction masks
y_test_en, y_pred_en = encode_masks_to_rgb(y_test, y_pred, color_map)

PATH_RESULTS = os.path.join(cfg.DIRS.results, model_name)
clean_folder(PATH_RESULTS)

save_predictions(X_test, 
                 y_test_en,
                 y_pred_en,
                 PATH_RESULTS,
                 imgs_labels,
                 confusion_classes,
                 color_map)

Find confusion matrix as well as prediction masks in `results` folder.